In [ ]:
from qiskit import *
import qiskit
import numpy as np

In [6]:
from qiskit.circuit.library import *
import matplotlib.pyplot as plt 

In [ ]:
#Test
qc=QuantumCircuit(2)
qc.append(HGate(),[0])
qc.append(HGate(),[1])
qc.draw()

┌───┐
q_0: ┤ H ├
     ├───┤
q_1: ┤ H ├
     └───┘

In [6]:
from qiskit_ibm_runtime import QiskitRuntimeService
service=QiskitRuntimeService(channel='ibm_quantum',token='4dbb00cf62afaefcd5a75717b4366b6ede70c230cd7a8524813b6d70dda1067cd9d975b70a7d6d673b47367b4a0e508bcc57702ee6cfcbb6f414da06d1f01596')

In [7]:
backend=service.backend(name='ibm_brisbane')

In [ ]:
from qiskit_ibm_runtime import SamplerV2 as Sampler
sampler=Sampler(backend)
job=sampler.run(qc)
print(job.job_id())

In [ ]:
def grover_oracle(n, marked_state):
    oracle=QuantumCircuit(n)
    #Flipping qubits which '0' in marked state
    for i, bit in enumerate(reversed(marked_state)):
        if bit=='0':
            oracle.x(i)
    #Applying a multi-controlled Z gate
    if n==1:
        oracle.z(0)
    else:
        oracle.h(n-1)
        oracle.mcx(list(range(n-1)),n-1)
        oracle.h(n-1)
    #Reverting the X gates
    for i, bit in enumerate(reversed(marked_state)):
        if bit=='0':
            oracle.x(i)
    #Converting the oracle circuit into a gate.
    oracle_gate=oracle.to_gate()
    oracle_gate.name="Oracle"
    decomposed_circuit=oracle_gate.decompose(recurse=True)
    print(decomposed_circuit.qasm())
    return oracle_gate

In [ ]:
def diffuser(n):
    qc=QuantumCircuit(n)
    qc.h(range(n))
    qc.x(range(n))
    qc.h(n-1)
    qc.mcx(list(range(n-1)), n-1)
    qc.h(n-1)
    qc.x(range(n))
    qc.h(range(n))
    diffuser_gate=qc.to_gate()
    diffuser_gate.name="Diffuser"
    return diffuser_gate

In [ ]:
from qiskit_aer import Aer
from qiskit import transpile
n=3  #Assigning number of qubits
marked_state='101'  #Target state

#Describing a quantum circuit with n qubits and n classical bits
grover_circuit=QuantumCircuit(n, n)

#Initializing all qubits in equal superposition
grover_circuit.h(range(n))

#Optimal number of Grover iterations calculations
iterations=int(np.floor(np.pi/4*np.sqrt(2**n)))
print("Number of iterations:",iterations)

#Applying Grover iterations
for _ in range(iterations):
    grover_circuit.append(grover_oracle(n,marked_state),range(n))
    grover_circuit.append(diffuser(n),range(n))

#Measuring all qubits
grover_circuit.measure(range(n),range(n))

#Graphing the circuit
grover_circuit.draw(output='mpl')

#Running the circuit on the qasm simulator initiated in the beginning
backend=Aer.get_backend('qasm_simulator')
job=backend.run(grover_circuit, backend,shots=1024)
result=job.result()
counts=result.get_counts()

print("Measurement counts:", counts)
